# Some Tricks
Not everything you can do in a notebook works properly when shared over `jupyterlab-dat`, but there are some pretty cool things that do work.

In [44]:
import IPython

# Styling for presentation

There's a lot going on in Lab by default: you may want to turn some of the UI noise down.

In [51]:
IPython.display.HTML("""
<style>
#main .jp-InputPrompt,
#main .jp-OutputPrompt { opacity: 0.5; position:absolute; z-index: 999; right: 0;}
#main #jp-top-panel:not(:hover),
#main #jp-bottom-panel:not(:hover),
#main .jp-SideBar:not(:hover),
#main .jp-Toolbar:not(:hover)
{opacity: 0.1;}
</style>
""")

## DataFrames

In [52]:
import pandas
df = pandas.util.testing.makeDataFrame()
df.head()

,A,B,C,D
9bg8RGqAy4,1.486843,-0.594134,0.919313,1.220785
KiSRdeAFgV,2.127711,0.279816,1.265447,-2.295364
JeYLPENO30,-0.765905,-1.195798,-0.843376,-0.535100
j0yIWhlRoM,-0.591685,-1.380916,0.381003,-1.437396
e8pIx9ADwc,-0.100585,-0.003973,-0.511177,-0.290147


## Plotting

In [ ]:
%matplotlib inline
df.plot.hist();

# VDOM
[vdom](https://github.com/nteract/vdom) has almost the same expressive power as widgets, but at least part of it doesn't use comm.

In [41]:
import vdom as h
some_vdom = h.h1("Hello VDOM 👋")
a_vdom_handle = display(some_vdom, display_id=True)

<h2>Buh-bye! 👋 </h2>

In [42]:
a_vdom_handle.update(h.h2("Buh-bye! 👋 "));

# Command Linking
You can publish URLs that enable JupyterLab commands:

In [43]:
%%html
<h1><a href="https://jupyter.org" data-commandlinker-command="notebook:restart-run-all">Restart & Run All</a></h1>

# One-click URL
If you are using a JupyterHub (like [Binder](https://mybinder.org)), you can create clickable links that will launch right into a shared stream. While _widgets don't work_ without the end user running our notebook, they are pretty handy for some quick UI.

In [53]:
import re, io, urllib.parse, qrcode.image.svg, ipywidgets as W, traitlets as T

In [78]:
@W.interact()
def make_link_and_qr(
    hub = W.Text("https://mybinder.org/v2/gh/"), repo = W.Text("deathbeds/jupyterlab-dat"), branch = W.Text("master"),
    dat = W.Textarea(placeholder="dat://", rows=4), color = W.ColorPicker(value="#2aca4b"), size = W.IntSlider(20, min=10, max=100),
):
    url = f"{hub}{repo}/{branch}?urlpath=lab/tree/dat-mkii/{dat.replace('dat://', '')}"
    qr = qrcode.QRCode(box_size=size)
    qr.add_data(url)
    qr_img = qr.make_image(qrcode.image.svg.SvgPathImage)
    out = io.BytesIO()
    qr_img._write(out)
    svg = re.sub(
        '<svg.*?viewbox',
        '<svg viewbox',
        out.getvalue().decode("utf-8")
    )
    svg = re.sub('#000000', color, svg)
    svg_url = f"data:image/svg+xml;utf-8,{urllib.parse.quote(svg)}"
    
    return IPython.display.HTML(f"""
        <a href="{url}" style="text-decoration: underline; font-size: 2rem; color: {color};">DRAG DAT ⬡</a>
        <br/>
        <i>or Ctrl+Shift click, Copy Link Address</i>
        <br/>
        <code>{url}</code>
        <br/>
        <a href="data:image/svg+xml;{svg_url}"><img src="{svg_url}" /></a>
    """)

interactive(children=(Text(value='https://mybinder.org/v2/gh/', description='hub'), Text(value='deathbeds/jupy…

You _can_, however, show the last known state of a `ipywidgets.interact`

In [75]:
[IPython.display.display(out["data"], raw=True) 
 for out in 
 make_link_and_qr.widget.children[-1].outputs];